## 訓練  
Model.pyで設定したCNNモデルと前処理で作成したNamPy配列から訓練を行う。

In [ ]:
import tensorflow as tf
import logging
import numpy as np
from sklearn.model_selection import train_test_split

# cnnモデルのインポート
from Model import cnn_model_fn

## モデルの設定

In [ ]:
model_dir = './model/etl8g_convnet_model'

進捗を追跡できるようにロギングの設定

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log,
    every_n_iter=50)
summary_hook = tf.train.SummarySaverHook(
    save_steps=500,
    output_dir=model_dir,
    scaffold=tf.train.Scaffold(summary_op=tf.summary.merge_all()))

データを読み込み、訓練データ・テストデータに分ける。

In [ ]:
hira = np.load('hira.npz')
train_data, eval_data, train_labels, eval_labels = train_test_split(
    hira['data'], hira['labels'], stratify=hira['labels'], random_state=0)

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

モデルの訓練  
30000回訓練を行うが、500ステップごとに60回に分けることで、学習させながら精度を追うことができる。

In [ ]:
etl8g_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=model_dir)

In [ ]:
for i in range(60):
    etl8g_classifier.train(
        input_fn=train_input_fn,
        steps=500,
        hooks=[logging_hook, summary_hook])

    eval_results = etl8g_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

TensorBoardで確認すると、精度の変化は  
<img src="https://i.imgur.com/AlI16sR.png">  
となり、このまま続けても99%以上にはならないと考えられる。  
これ以上精度をあげるにはデータの画素数とサンプル数を増やす、層の複雑化、パラメータの調節などが挙げられる。